In [ ]:
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error, IntegrityError
from dotenv import load_dotenv
import os

load_dotenv()

### Criando conexão com o MYSQL usando as variaveis usadas no .env

In [ ]:
config = {
    'user': os.getenv("user"),
    'password': os.getenv("password"),
    'host': os.getenv("host"),
}

try:
    conn = mysql.connector.connect(**config)
    print("Conexão estabelecida com sucesso!")
    
except mysql.connector.Error as error:
    # Erro ao estabelecer a conexão
    print(f"Erro ao conectar ao banco de dados: {error}")
    
cursor = conn.cursor()

### Criacao do banco de dados

In [ ]:
with open('../script_criacao.sql', 'r') as file:
    sql_script = file.read()

for result in cursor.execute(sql_script, multi=True):
    pass
conn.commit()

### Carregando todos as dataframes normalizados

In [ ]:
df = pd.read_csv("../../dados_normalizados/collections.csv")
df = df.replace({np.NaN : None})
df_movies = pd.read_csv("../../dados_normalizados/movies_base.csv")
df_movies = df_movies.replace({np.NaN : None})
df_diretores = pd.read_csv("../../dados_normalizados/diretores.csv")
df_diretores = df_diretores.replace({np.NaN: None})
df_atores = pd.read_csv("../../dados_normalizados/atores.csv")
df_atores = df_atores.replace({np.NaN: None})
df_elenco = pd.read_csv("../../dados_normalizados/elenco.csv")
df_elenco = df_elenco.replace({np.NaN: None})
df_direcao = pd.read_csv("../../dados_normalizados/direcao.csv")
df_direcao = df_direcao.replace({np.NaN: None})

### Inserir valores do dataframe diretores na Tabela

In [ ]:
for _, row in df_diretores.iterrows():
    idDiretor = row["id"]
    nome = row["name"]
    genero = row["gender"]
    
    try:
        query = "INSERT INTO diretor (idDiretor, genero, nome) VALUES (%s, %s, %s)"
        values = (idDiretor,genero,nome)
        cursor.execute(query, values)
        conn.commit()
    except IntegrityError as e:
        print(f"Erro de ID duplicado: {e}")
        conn.rollback()
    except Error as e:
        print(f"Erro ao inserir dados: {e}")
        conn.rollback()

### Inserir valores do dataframe atores na tabela

In [ ]:
for _, row in df_atores.iterrows():
    idAtor = row["id"]
    nome = row["name"]
    genero = row["gender"]
    
    try:
        query = "INSERT INTO ator (idAtor, genero, nome) VALUES (%s, %s, %s)"
        values = (idAtor,genero,nome)
        cursor.execute(query, values)
        conn.commit()
    except IntegrityError as e:
        print(f"Erro de ID duplicado: {e}")
        conn.rollback()
    except Error as e:
        print(f"Erro ao inserir dados: {e}")
        conn.rollback()

### Inserir valores do dataframe coleções na tabela

In [ ]:
for i, x in df.iterrows():
    id = x['id']
    nome = x['nome']

    try:
        query = "INSERT INTO colecao (idColecao, nome) VALUES (%s, %s)"
        values = (id, nome)
        cursor.execute(query, values)
        conn.commit()
        
    except IntegrityError as e:
        print(f"Erro de ID duplicado: {e}")
        conn.rollback()
    except Error as e:
        print(f"Erro ao inserir dados: {e}")
        conn.rollback()

### Inserir valores do dataframe filmes na tabela

In [ ]:
for i,row in df_movies.iterrows():
    idColecao = row["idColecao"]
    if idColecao != None:
        idColecao = int(idColecao)
    custo = row["custo"]
    idFilme = row["idFilme"]
    lancamento = row["lancamento"]
    bilheteria = row["bilheteria"]
    duracao = row["duracao"]
    titulo = row["titulo"]
    nota = row["nota"]
    
    try:
        query = "INSERT INTO filme (idFilme, titulo, bilheteria, custo, lancamento, nota, duracao, idColecao) VALUES (%s, %s, %s, %s, %s, %s,%s, %s)"
        values = (idFilme, titulo, bilheteria, custo, lancamento, nota, duracao, idColecao)
        cursor.execute(query, values)
        conn.commit()
    except IntegrityError as e:
        print(f"Erro de ID duplicado: {e}")
        conn.rollback()
    except Error as e:
        print(f"Erro ao inserir dados: {e}")
        conn.rollback()

### Elencar diretores com seus filmes

In [ ]:
for _,row in df_direcao.iterrows():
    idDiretor= row["idDiretor"]
    idFilme= row["idFilme"]
    
    try:
        query = "INSERT INTO direcao (idDiretor, idFilme) VALUES (%s, %s)"
        values = (int(idDiretor), int(idFilme))
        cursor.execute(query, values)
        conn.commit()
    except IntegrityError as e:
        print(f"Erro de ID duplicado: {e}")
        conn.rollback()
    except Error as e:
        print(f"Erro ao inserir dados: {e}")
        conn.rollback()

### Elencar Atores com seus filmes

In [ ]:
for _,row in df_elenco.iterrows():
    idAtor= row["idAtor"]
    idFilme= row["idFilme"]
    
    try:
        query = "INSERT INTO elenco (idAtor, idFilme) VALUES (%s, %s)"
        values = (int(idAtor), int(idFilme))
        cursor.execute(query, values)
        conn.commit()
    except IntegrityError as e:
        print(f"Erro de ID duplicado: {e}")
        conn.rollback()
    except Error as e:
        print(f"Erro ao inserir dados: {e}")
        conn.rollback()

### Finalizar conexão com o banco

In [ ]:
cursor.close()
conn.close()